In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random
import os

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [2]:
search = '국민연금'

In [3]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles_temp = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        # 각 페이지에서 URL이 10개가 아닌 경우 해당 페이지의 URL 출력
        if len(new_urls) % 10 != 0:
            print(f"Warning: {url} contains {len(new_urls)} links instead of 10.")
        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []
        current_titles = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press_name = article_infos[idx - 1]
                    
                    if "CHIEF EXECUTIVE" not in current_press_name:
                        current_press.append(current_press_name)
                        current_titles.append(article_titles_temp[len(current_dates)])
                        current_dates.append(match.group(0))

        articles.extend(new_urls[:len(current_titles)])  # articles length should match with titles
        titles.extend(current_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(1, 1.5))
        
    return articles, titles, dates, press

In [4]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time

In [6]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [5]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=400):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&sort=2&" + date_part + "&mynews=1&office_type=1&office_section_code=1&news_office_checked=1028&office_category=0&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [6]:
# search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")

KeyboardInterrupt: 

In [20]:
df

,date,time,title,content,press,link
0,2004.04.01,2004-04-01 18:29:00,"한나라, 고교평준화 사실상 포기",\n\t\t\t[한겨레] ■총선공약 발표 한나라당이 1일 고등학교 평준화를 사실상...,한겨레,https://n.news.naver.com/mnews/article/028/000...
1,2004.04.01,2004-04-01 19:55:00,"고려아연, 기관투자자에 항복","\n\t\t\t[한겨레] 속보=고려아연이 주주중시 경영, 투명 경영을 요구하는 시...",한겨레,https://n.news.naver.com/mnews/article/028/000...
2,2004.04.01,2004-04-01 19:56:00,“복지 위해선 정치·기업 민주화 선행돼야”,\n\n\n\n\n[한겨레] 학술지 ‘아세아연구’한국의 사회복지 집중해부 고려대 ...,한겨레,https://n.news.naver.com/mnews/article/028/000...
3,2004.04.13,2004-04-13 21:02:00,마스터플랜 기획단장 이필상씨,\n\n\n\n\n [한겨레] 보건복지부는 13일 이필상(57) 고려대 경영학과 ...,한겨레,https://n.news.naver.com/mnews/article/028/000...
4,2004.04.15,2004-04-15 22:58:00,인물/서초갑 이혜훈 ‘박풍’ 힘입어 열세 뒤집어,\n\n\n\n\n [한겨레] 서울 서초갑의 이혜훈(39·한나라당) 당선자는 미국...,한겨레,https://n.news.naver.com/mnews/article/028/000...
...,...,...,...,...,...,...
65,2004.07.21,2004-07-21 18:18:00,빈곤층 못낸 건보료 탕감해준다,\n\t\t\t [한겨레] 당정 생활안정 지원책 발표 생계가 어려운 저소득층이 ...,한겨레,https://n.news.naver.com/mnews/article/028/000...
66,2004.07.23,2004-07-23 18:48:00,“신용불량자 빚갚게 국민연금 환급”,\n\t\t\t [한겨레] 국민연금 일시반환금 제도를 이용해 소액 신용불량자들을 ...,한겨레,https://n.news.naver.com/mnews/article/028/000...
67,2004.07.27,2004-07-27 20:48:00,가구 58% “쓸 수 있는 돈 줄었다”,"\n\n\n\n\n [한겨레] ■상의, 1000가구 주부 조사 경기침체가 계속되...",한겨레,https://n.news.naver.com/mnews/article/028/000...
68,2004.07.28,2004-07-28 18:08:00,7월 29일 뉴스마당,\n\t\t\t [한겨레] 전력수요 또 사상최고치 경신 불볕더위에 따른 냉방기 ...,한겨레,https://n.news.naver.com/mnews/article/028/000...


In [22]:
df['content'][0]

'\n\t\t\t[한겨레] ■총선공약 발표  한나라당이 1일 고등학교 평준화를 사실상 포기하는 내용을 포함한 총선 공약을 내놓아 논란이 예상된다. 한나라당은 또 노인, 장애인 등 소외계층에 대해선 국가예산으로 연금 혜택을 주는 ‘전국민 1인1연금제 도입’도 공약에 담았다. ■ 고교 평준화 폐지 =  한나라당은 이날 발표한 공약에서 “고교 평준화는 상위 계층의 학력을 떨어뜨리고 학교 선택권을 제한하는 문제를 안고 있다”며 “희망하는 사학은 평준화 정책 대상에서 제외하겠다”고 밝혔다. 사립학교 비중이 70% 안팎에 이르는 현실을 감안할 때, 이는 고교 평준화제를 사실상 폐지하자는 것으로 풀이된다. 교육인적자원부 관계자는 이에 대해 “사립고의 경우 인건비를 100% 국가에서 지원받는 등 적지않은 공적 혜택을 받고 있다”며 “평준화 폐지를 통해 사립고에 학생 선발권을 주려면 이런 혜택도 같이 없애야 앞뒤가 맞는 것”이라고 지적했다.  ■   1인1연금 제도 = 한나라당은 국민연금제도를 기초연금과 소득비례연금으로 분리해, 기초연금을 전 국민을 대상으로 하는 1인1연금제로 바꾸겠다고 밝혔다. 이렇게 하면 전업주부, 노령층, 저소득층, 장애인, 비정규직 근로자까지 연금 혜택을 받을 수 있다는 것이다. 한나라당은 이를 위해 국가예산에서 연차적으로 재원을 확보해 장애인 등 소외계층 200만명에게 월30만원의 연금을 지원한다는 구상을 세워두고 있다. 특히 장애인의 경우 일반 수급자보다 수급연령을 앞당겨 적용하는 방안도 추진하겠다고 약속했다. ■  선심성 논란 =  한나라당은 이날 실효성이 높아 보이지 않는 정책들을 무더기로 공약에 담아, 총선을 앞둔 선심성 공약 아니냐는 비판을 받고 있다. 1인1연금제의 경우 전면 시행됐을 경우 연간 7조원의 예산을 필요로 하는 대규모 사업인데, 재원조달 방안에 대해선 막연하게 ‘국가예산’이라고만 밝히고 있다. 정밀한 검토를 거쳤는지 의문이다. 실업계 고교 무상교육 전면 실시, 재래시장 활성화를 위한 1조원 투입 5개년 계획 등에서도 재원은 대

In [36]:
# 데이터 프레임 저장
path = r"C:\\Users\\boyu571\\boyu571_Github\\01_Kakaobank_SKKU_Research_23\\data_gatherting"
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
full_path = os.path.join(path, filename)
df.to_csv(full_path, encoding='utf-8-sig', index=False)

In [74]:
# result = pd.concat([df1, df2, df3, df4], ignore_index=True)
# result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# result

,date,time,title,content,press,link
0,2021.02.01,2021-02-01 00:00:19,'극한의 롤러코스터'…게임스탑發 증시 폭락 우려 확산,\n'극한의 롤러코스터' 게임스탑 격변의 한주한주간 400% 폭등…AMC 등도 주가...,이데일리언론사 선정,https://n.news.naver.com/mnews/article/018/000...
1,2021.02.01,,"뉴욕증시 비트코인 거품 붕괴 조짐, 게임스톱 공매도 일파만파… 테슬라 애플...",,글로벌이코노믹,http://www.g-enews.com/
2,2021.02.01,,"'애로부부' 현영, 결혼생활 10년에도 남편과 뜨거운 이유? ""50살에 다른 매력...",,뉴스웍스,http://www.newsworks.co.kr/
3,2021.02.01,2021-02-01 02:01:33,마디지수 깨진 글로벌 증시…유동성 랠리 끝나나,"\n\t\t\t[이데일리 권소현 기자, 뉴욕=김정남 특파원] 사상 최고치 경신을 거...",A1면 TOP,https://n.news.naver.com/mnews/article/018/000...
4,2021.02.01,2021-02-01 03:03:03,[밀레니얼 톡] 당당하게 ‘돈 이야기’ 하며 삽시다,\n\t\t\t\t\t\t\t\t\t\t90년대생 세 명을 인터뷰했다. 주제는 ‘주...,A29면 1단,https://n.news.naver.com/mnews/article/023/000...
...,...,...,...,...,...,...
12658,2021.03.31,,인도가 '비트코인 금지법'을 도입하면?,,코인데스크코리아,https://www.coindeskkorea.com/
12659,2021.03.31,,[업비트 특징주]비트토렌트 코인 주가 10% 이상 상승,,국제뉴스,http://www.gukjenews.com/
12660,2021.03.31,,트리핀의 딜레마,,단비뉴스,http://www.danbinews.com
12661,2021.03.31,,NFT,,단비뉴스,http://www.danbinews.com


In [75]:
# df.to_csv('비트코인_20210201_20210331.csv', encoding='utf-8-sig', index=False)